In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

cryto_dict={}

urls=[f'https://coinmarketcap.com/?page={i}' for i in range(1,4)]

for cur_url in urls:
    data_fet = requests.get(cur_url)
    soup=BeautifulSoup(data_fet.text,"html.parser")

    # Extract all coin names based on the inspected class
    names = soup.find_all('p', class_='sc-65e7f566-0 iPbTJf coin-item-name')
    cryto_dict["coin_names"] = [name.text.strip() for name in names]

    symbols = soup.find_all('p', class_='sc-65e7f566-0 byYAWx coin-item-symbol')
    # Clean and collect text values
    cryto_dict["coin_symbols"] = [symbol.text.strip() for symbol in symbols]

    rows = soup.find_all('tr')
    prices = []
    for row in rows:
        tds = row.find_all('td')
        if len(tds) > 3:  # check if this row has price column
            price = tds[3].text.strip()  # 4th td is price based on page structure
            prices.append(price)
    cryto_dict["Price"]=prices


    # Extract 1h change values
    changes_1h = soup.find_all('span', class_='sc-1e8091e1-0 jgYsZM')
    # Clean and collect text values
    cryto_dict["change_1h%"] = [change.text.strip() for change in changes_1h]


    # Extract market cap values
    market_caps = soup.find_all('span', class_='sc-11478e5d-1 jfwGHx')
    # Clean and collect text values
    cryto_dict["coin_market_cap"] = [cap.text.strip() for cap in market_caps]


    volumes = []
    for row in rows:
        tds = row.find_all('td')
        if len(tds) > 8:
            volume_container = tds[8]
            dollar_value_tag = volume_container.find('a').find('p')
            if dollar_value_tag:
                volumes.append(dollar_value_tag.text.strip())
            else:
                volumes.append('N/A')
    cryto_dict["volume_24hrs"]=volumes


    # Extract supply values
    supplies = soup.find_all('div', class_='circulating-supply-value')
    # Clean and collect text values
    cryto_dict["circulatory_supplies"] = [supply.find('span').text.strip() for supply in supplies]

    min_len = min(len(v) for v in cryto_dict.values())
    for k in cryto_dict:
        cryto_dict[k] = cryto_dict[k][:min_len]


df=pd.DataFrame(cryto_dict)
df.shape

(10, 7)

In [6]:
df

,coin_names,coin_symbols,Price,change_1h%,coin_market_cap,volume_24hrs,circulatory_supplies
0,Fasttoken,FTN,$4.45,0.04%,"$1,944,063,371","$57,668,900",436.26M
1,Jupiter Perps LP,JLP,$4.34,0.65%,"$1,492,378,268","$16,614,491",343.49M
2,Zeebu,ZBU,$3.27,1.06%,"$847,203,483","$503,389",258.97M
3,Fellaz,FLZ,$1.42,0.14%,"$673,700,222","$2,592,274",471.87M
4,Usual USD,USD0,$0.9975,0.30%,"$652,215,598","$5,315,752",653.81M
5,Ondo US Dollar Yield,USDY,$1.09,0.03%,"$635,416,885","$2,312,347",581.35M
6,Falcon USD,USDf,$0.9997,0.19%,"$563,485,757","$2,815,017",563.65M
7,Loaded Lions,LION,$0.01656,3.49%,"$549,397,153","$168,062",33.16B
8,would,WOULD,$0.5001,0.03%,"$499,872,289","$152,361",999.45M
9,WhiteRock,WHITE,$0.0007299,0.00%,"$474,445,028","$775,018",650B
